# Pre-market Sythetic Orderbook (Demo)

### Import packages

In [79]:
# import importlib
import requests
import pandas as pd
import time
import math


In [2]:
def get_whales_pre_market_currencies() -> pd.DataFrame:
    # def get_whales_pre_market_currencies() -> list[str]:
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
        'if-none-match': 'W/"5f0d-8HSEDhAaRqqgDuK6HrpWGMyHKbk"',
        'origin': 'https://pro.whales.market',
        'priority': 'u=1, i',
        'referer': 'https://pro.whales.market/',
        'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    }

    response = requests.get(
        'https://api-v2.whales.market/v2/tokens?take=100&page=1&type=pre_market&search=&category=pre_market&statuses[]=active&statuses[]=settling&sort_vol=desc',
        headers=headers,)

    if response.status_code != 200:
        raise Exception('Request Error')

    j = response.json()
    # print(j)
    return pd.DataFrame(j['data']['list'])
    # return [curr['symbol'] for curr in j['data']['list']]

In [3]:
def get_gateio_pre_market_currencies() -> pd.DataFrame:
    gateio_premarket_currencies_url = 'https://www.gate.io/apiw/v2/pre_market/currencies?type=1&page=1&limit=99'
    response = requests.get(gateio_premarket_currencies_url)

    if response.status_code != 200:
        raise Exception('Request Error')

    j = response.json()
    df = pd.DataFrame(j['data']['list'])

    return df



In [4]:
def get_bybit_pre_market_currencies() -> pd.DataFrame:
    cookies = {
        '_abck': 'B796D0364EFB4118DD46CE1BC906E381~0~YAAQxetGaHUqlVGPAQAAM+9Hjwua4sJdo6LUgbw1O0Gy9ugKCe/I6fxmK/bIR9Tz/vaCmXiBOewvfn1iMf8kUenelsjR498rwkFRYjIkWcq3Q7WzGClNzvrdJ7fUguDSaFqj5/7cSLhY0mqzJ2Qo4eJgh1GCmyOflUfDKXSsEVdBlsQK5tW2xrrKZPH7uxdws28CVgGCh99lahAFFf6HO3eJEvn1ywnFtDuJ2TMzpjnwqcOVtIGNDoLcQTSKOk30JZHYoSXupwBk6HE9i+QS2YGxq9ryXbdW59gEOSREBge94TvFWs3rsUmW7fwDuhOP4AQqW7ysAGN0HS2Az5Uqh1JIsVdA/hZQrhGEWb6Ff035xEpg8BWdh0mPRsti~-1~-1~-1',
    }

    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en',
        'dnt': '1',
        'lang': 'en',
        'origin': 'https://www.bybit.com',
        'priority': 'u=1, i',
        'referer': 'https://www.bybit.com/',
        'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'traceparent': '00-f355bb1ea82b8685509918c86a6729db-e548f6630a12a96e-00',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    }

    response = requests.get(
        'https://api2.bybit.com/spot/api/marketTrading/v1/token/tokenList', cookies=cookies, headers=headers)

    if response.status_code != 200:
        raise Exception('Request Error')

    j = response.json()
    df = pd.DataFrame(j['result'])

    return df




In [5]:
def get_whales_pre_market_orderbook(token: str, side: str) -> pd.DataFrame:
    if side != 'buy' and side != 'sell':
        raise Exception('side should be either buy or sell')

    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
        'if-none-match': 'W/"1eb3-hannLXcIpqHTs8gMYRF9uJAYOQE"',
        'origin': 'https://pro.whales.market',
        'priority': 'u=1, i',
        'referer': 'https://pro.whales.market/',
        'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    }

    params = {
        'take': '30',
        'page': '1',
        'type': side,
        'full_match': '',
        'symbol': token,
        'status': 'open',
        'min_price': '',
        'max_price': '',
        'sort_price': 'ASC',
        'category_token': 'pre_market',
        # 'chains': '666666',
    }

    response = requests.get(
        'https://api-v2.whales.market/v2/offers', params=params, headers=headers)

    if response.status_code != 200:
        raise Exception('Error')

    j = response.json()
    # print(j['data'])
    # obs =

    return pd.DataFrame(j['data']['list'])




In [6]:
def get_bybit_pre_market_orderbook(token: str, side: str) -> pd.DataFrame:
    if side == 'sell':
        order_type = '1'

    if side == 'buy':
        order_type = '2'

    cookies = {
        '_abck': 'B796D0364EFB4118DD46CE1BC906E381~0~YAAQhOtGaOkusU+PAQAA2AAWjwuVOFcNCJVSiQAUh5WTv6er5VRJ5iMgdAiKcHfZVHviVJjIyd3LyWAHQ6jw82SEjJuyQIV6E0KdDet8aLbSHtZ4xGb1RWkeHzpjUXrPIJUwpVZT0QSaKjqhJ07MYj3kDzIuGKjTR7pat9zlBLmyPhS16KN789Zdjz6tzjMTz1gR2rLvU18W51TQd4bD8hk9Hj213lEstV8sCrMrj8xoIXF1RXYdTJObpEaN8RPpr8DdyAp3/fEXOnMJEE2Fgx8xQRTPYbhm7BvjvBr0C0f14oEwdcgDcEZKp65Fa95/JkMAhButrZ6CyVhB90Z4/yuGFSS92M99wlQiz8E+6efog72zvwE2ZU3lFuAPDpr0sRB5VMSBRWw=~-1~-1~-1',
    }

    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en',
        'dnt': '1',
        'lang': 'en',
        'origin': 'https://www.bybit.com',
        'priority': 'u=1, i',
        'referer': 'https://www.bybit.com/',
        'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'traceparent': '00-bcc60bf8778f864e0c0b82c1903f792f-cae725ec958abdce-00',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    }

    params = {
        'minPrice': '',
        'maxPrice': '',
        'minAmount': '',
        'maxAmount': '',
        'showMyOrder': '0',
        'matchUserAsset': '0',
        'orderType': order_type,
        'tokenName': token,
        'pageSize': '11',
    }

    response = requests.get(
        url='https://api2.bybit.com/spot/api/marketTrading/v1/order/tradeOrders',
        params=params,
        cookies=cookies,
        headers=headers,
    )

    if response.status_code != 200:
        raise Exception('Error')

    j = response.json()
    obs = pd.DataFrame(j['result'])

    return obs




In [7]:
def get_gateio_pre_market_orderbook(token: str, side: str) -> pd.DataFrame:
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en,en-US;q=0.9,zh-HK;q=0.8,zh-TW;q=0.7,zh;q=0.6',
        'baggage': 'sentry-environment=production,sentry-release=2w8LYBYs_SMvIiAIlusi_,sentry-public_key=49348d5eaee2418db953db695c5c9c57,sentry-trace_id=54263241e3ae4c5cb2c8d26efe4b4440',
        'csrftoken': '1',
        'dnt': '1',
        'priority': 'u=1, i',
        'referer': f'https://www.gate.io/en/pre-market/{token}',
        'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'sentry-trace': '54263241e3ae4c5cb2c8d26efe4b4440-9d40373b52fccc98-0',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    }

    params = {
        'page': '1',
        'limit': '10',
        'currency': token,
        'side': side,
        'status': 'no_transaction',
    }

    response = requests.get(
        'https://www.gate.io/apiw/v2/pre_market/market_orders', params=params, headers=headers)

    if response.status_code != 200:
        raise Exception('Error')

    j = response.json()

    if j.get('message') == 'success':
        data_list = j['data']['list']
        obs = pd.DataFrame(data_list)
        return obs




In [8]:
def convert_orderbook_tick(orderbook_df: pd.DataFrame, exchange: str, spread_factor=1):
    quotes = []
    orders = orderbook_df.to_dict(orient='records')

    if exchange == 'bybit':
        for order in orders:
            quotes.append({
                'raw_price': order['price'],
                'price': order['price'] * spread_factor,
                'qty': order['quantity'],
                'unfilled_qty': order['quantity'],
                'unfilled_order_amt':order['orderAmount'],
                'order_amt': order['orderAmount'],
                'exch': 'bybit',
                'spread': spread_factor-1
            })
        return quotes
    # elif exchange == 'gateio':
    #     for order in orders:
    #         quotes.append({
    #             'price': order['price'] * spread_factor,
    #             'quantity': order['amount'],
    #             'order_amount': order['order_value'],
    #             'exchange': 'gateio',
    #         })
    #     return quotes
    elif exchange == 'whales':
        for order in orders:
            quotes.append({
                'raw_price': order['price'] * order['ex_token__price'],
                'price': order['price'] * spread_factor * order['ex_token__price'],
                'qty': order['total_amount'],
                'unfilled_qty': order['total_amount'] - order['filled_amount'],
                'order_amt': (order['total_amount'] - order['filled_amount']) * order['price'] * spread_factor * order['ex_token__price'],
                # 'total_order_amt': order['value'] * order['ex_token__price'],
                'collateral_quoted_in': order['ex_token__symbol'],
                'exch': 'whales', 
                'spread': spread_factor-1
            })
        return quotes
    else:
        raise ValueError('Exchange not supported.')

### Get the available currencies and the common tokens

In [9]:
def get_available_currencies() -> list[str]: 
  whales_tokens = get_whales_pre_market_currencies()
  bybit_tokens = get_bybit_pre_market_currencies()

  curr_time = time.time()
  bybit_tokens = bybit_tokens[(bybit_tokens['tradeEndTime'] > curr_time) | (bybit_tokens['tradeEndTime'] == 0)]

  # common_tokens = list(set(gateio_tokens['currency'].unique().tolist()) & set(bybit_tokens['tokenName'].unique().tolist()))
  common_tokens = list(set(whales_tokens['symbol'].unique().tolist()) & set(
      bybit_tokens['tokenName'].unique().tolist()))
  # print('common_tokens = ', common_tokens)
  return common_tokens


In [14]:
get_available_currencies()

common_tokens =  ['BLAST', 'ZRO', 'ZKL', 'EIGEN']


['BLAST', 'ZRO', 'ZKL', 'EIGEN']

### Get the snapshot of the 'buys' & 'sells' from both exchanges

In [17]:

# token = common_tokens[0]
token = 'ZRO'

#fetch and clean orderbooks
whales_bids = get_whales_pre_market_orderbook(token, 'buy')
whales_asks = get_whales_pre_market_orderbook(token, 'sell')
whales_bids = whales_bids[whales_bids['filled_amount'] <= 0.9*whales_bids['total_amount']]
whales_asks = whales_asks[whales_asks['filled_amount'] <= 0.9*whales_asks['total_amount']]
bybit_bids = get_bybit_pre_market_orderbook(token, 'buy')
bybit_asks = get_bybit_pre_market_orderbook(token, 'sell')
bybit_bids['price'] = [float(p) for p in bybit_bids['price']]
bybit_asks['price'] = [float(p) for p in bybit_asks['price']]
bybit_bids['quantity'] = [float(p) for p in bybit_bids['quantity']]
bybit_asks['quantity'] = [float(p) for p in bybit_asks['quantity']]

# Combine orderbooks of bid and ask from 2 exchagnes
agg_bids = convert_orderbook_tick(bybit_bids, 'bybit', .98) + convert_orderbook_tick(whales_bids, 'whales', .975)
agg_asks = convert_orderbook_tick(bybit_asks, 'bybit', 1.02) + convert_orderbook_tick(whales_asks, 'whales', 1/.975)
agg_bids_df = pd.DataFrame(agg_bids)
agg_asks_df = pd.DataFrame(agg_asks)
agg_bids_df['collateral_quoted_in'] = agg_bids_df['collateral_quoted_in'].fillna("USDT")
agg_asks_df['collateral_quoted_in'] = agg_asks_df['collateral_quoted_in'].fillna("USDT")
agg_bids_df.sort_values(by=['price'], ascending=False, inplace=True, ignore_index=True)
agg_asks_df.sort_values(by=['price'], ascending=True, inplace=True, ignore_index=True)

orb = pd.concat([
    agg_bids_df.rename(columns={c: f'{c}_b' for c in agg_asks_df.columns}),
    agg_asks_df.rename(columns={c: f'{c}_a' for c in agg_asks_df.columns})],
    axis=1)

price_diff = orb['price_b'][0] - orb['price_a'][0]
# if price_diff < 0:
#   continue

Put in demo data

In [49]:
orb = pd.read_csv('./EIGEN_orderbook_testdata.csv')

In [50]:
orb.head()

,raw_price_b,price_b,qty_b,order_amt_b,exch_b,spread_b,collateral_quoted_in_b,raw_price_a,price_a,qty_a,order_amt_a,exch_a,spread_a,collateral_quoted_in_a
0,7.72,7.57,33.0,254.76,bybit,-0.02,USDT,7.30,7.49,183.00,1370.15,whales,0.03,USDT
1,7.71,7.56,2.0,15.42,bybit,-0.02,USDT,7.38,7.57,110.00,832.39,whales,0.03,ETH
2,7.71,7.56,2.0,15.42,bybit,-0.02,USDT,7.50,7.69,34.65,266.54,whales,0.03,USDC
3,7.71,7.56,2.0,15.42,bybit,-0.02,USDT,7.50,7.69,1350.00,10153.85,whales,0.03,USDT
4,7.71,7.56,2.0,15.42,bybit,-0.02,USDT,7.56,7.75,1115.00,8640.11,whales,0.03,ETH


### Merge & combine it as a table

In [51]:
orb[['exch_b', 'spread_b', 'raw_price_b', 'collateral_quoted_in_b', 'order_amt_b', 'qty_b', 'price_b', 'price_a', 'qty_a', 'order_amt_a', 'collateral_quoted_in_a', 'raw_price_a', 'spread_a', 'exch_a']]

,exch_b,spread_b,raw_price_b,collateral_quoted_in_b,order_amt_b,qty_b,price_b,price_a,qty_a,order_amt_a,collateral_quoted_in_a,raw_price_a,spread_a,exch_a
0,bybit,-0.02,7.72,USDT,254.76,33.0,7.57,7.49,183.00,1370.15,USDT,7.30,0.03,whales
1,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.57,110.00,832.39,ETH,7.38,0.03,whales
2,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.69,34.65,266.54,USDC,7.50,0.03,whales
3,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.69,1350.00,10153.85,USDT,7.50,0.03,whales
4,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.75,1115.00,8640.11,ETH,7.56,0.03,whales
5,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,8.21,1950.00,16000.00,USDT,8.00,0.03,whales
6,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,8.21,25.00,205.13,USDC,8.00,0.03,whales
7,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,8.49,100.00,832.00,USDT,8.32,0.02,bybit
8,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,8.51,10.00,83.40,USDT,8.34,0.02,bybit
9,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,8.51,30.00,250.20,USDT,8.34,0.02,bybit


## See arb

In [52]:
def fetch_network_name(token)->str:
  headers = {
      'accept': 'application/json, text/plain, */*',
      'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
      'if-none-match': 'W/"4d2-9L+LvMdAVw215IEGIl5WjHys9Z8"',
      'origin': 'https://pro.whales.market',
      'priority': 'u=1, i',
      'referer': 'https://pro.whales.market/',
      'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-site',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
  }

  response = requests.get(
      f'https://api-v2.whales.market/v2/tokens/detail/{token}', headers=headers)
  
  if response.status_code != 200:
      raise Exception('Error')
    
  j = response.json() 
  return j['data']['network_name']


In [53]:
def network_name_to_gas(network_name): 
  # Each tx cost .003005 SOL gas fee (.444 U eqv)
  if network_name == 'Solana':
    fetch_sol = requests.get("https://api.binance.com/api/v3/ticker/price?symbol=SOLUSDT")
    if fetch_sol.status_code != 200:
        raise Exception('Error')
    return float(fetch_sol.json()['price'])*.003005
  
  # Each tx cost .00076723ETH gas fee (2.75U eqv)
  elif network_name == 'Ethereum':
    fetch_eth = requests.get(
        "https://api.binance.com/api/v3/ticker/price?symbol=ETHUSDT")
    if fetch_eth.status_code != 200:
        raise Exception('Error')
    return float(fetch_eth.json()['price'])*.00076723
  
  # ################## SEND A TG MSG for a new chain
  return 0


# network_name_to_gas('Solana')

In [54]:
orders_to_be_taken = pd.DataFrame(columns=['exch_b', 'collateral_quoted_in_b', 'order_amt_b', 'price_b','qty' ,'price_a', 'order_amt_a', 'collateral_quoted_in_a', 'exch_a','gas', 'profit'])
bids = orb[['exch_b', 'spread_b', 'raw_price_b','collateral_quoted_in_b', 'order_amt_b', 'qty_b', 'price_b']].dropna(how='all')
asks = orb[['price_a', 'qty_a', 'order_amt_a', 'collateral_quoted_in_a', 'raw_price_a', 'spread_a', 'exch_a']].dropna(how='all')

In [55]:
row_b = row_a = 0
max_row_b = len(bids.index)
max_row_a = len(asks.index)

In [56]:
orb[['exch_b', 'spread_b', 'raw_price_b', 'collateral_quoted_in_b', 'order_amt_b', 'qty_b', 'price_b',
    'price_a', 'qty_a', 'order_amt_a', 'collateral_quoted_in_a', 'raw_price_a', 'spread_a', 'exch_a']].head()

,exch_b,spread_b,raw_price_b,collateral_quoted_in_b,order_amt_b,qty_b,price_b,price_a,qty_a,order_amt_a,collateral_quoted_in_a,raw_price_a,spread_a,exch_a
0,bybit,-0.02,7.72,USDT,254.76,33.0,7.57,7.49,183.00,1370.15,USDT,7.30,0.03,whales
1,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.57,110.00,832.39,ETH,7.38,0.03,whales
2,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.69,34.65,266.54,USDC,7.50,0.03,whales
3,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.69,1350.00,10153.85,USDT,7.50,0.03,whales
4,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,7.75,1115.00,8640.11,ETH,7.56,0.03,whales


In [57]:
bids.loc[0, 'price_b'] = 10
bids.loc[0, 'qty_b'] = 200
bids.loc[0,'order_amt_b'] = 2000

In [58]:
bids.head()

,exch_b,spread_b,raw_price_b,collateral_quoted_in_b,order_amt_b,qty_b,price_b
0,bybit,-0.02,7.72,USDT,2000.00,200.0,10.00
1,bybit,-0.02,7.71,USDT,15.42,2.0,7.56
2,bybit,-0.02,7.71,USDT,15.42,2.0,7.56
3,bybit,-0.02,7.71,USDT,15.42,2.0,7.56
4,bybit,-0.02,7.71,USDT,15.42,2.0,7.56


In [59]:
asks.head()

,price_a,qty_a,order_amt_a,collateral_quoted_in_a,raw_price_a,spread_a,exch_a
0,7.49,183.00,1370.15,USDT,7.30,0.03,whales
1,7.57,110.00,832.39,ETH,7.38,0.03,whales
2,7.69,34.65,266.54,USDC,7.50,0.03,whales
3,7.69,1350.00,10153.85,USDT,7.50,0.03,whales
4,7.75,1115.00,8640.11,ETH,7.56,0.03,whales


In [60]:
token = 'EIGEN'
route = ()
total_qty = 0.0
total_profit = 0.0
bids['to_be_taken'] = [False for _ in range(max_row_b)]
asks['to_be_taken'] = [False for _ in range(max_row_a)]

while row_b < max_row_b and row_a < max_row_a: 
  print((row_b, row_a))
  price_diff = bids['price_b'][row_b] - asks['price_a'][row_a]
  if price_diff <= 0:
    break
  print('price_diff', price_diff)
  
  marginal_gas = 0
  if bids['to_be_taken'][row_b] == False:
    marginal_gas += network_name_to_gas(fetch_network_name(token)) if bids['exch_b'][row_b] == 'whales' else 0 
  if asks['to_be_taken'][row_a] == False:
    marginal_gas = network_name_to_gas(fetch_network_name(token)) if asks['exch_a'][row_a] == 'whales' else 0
  print('marginal_gas', marginal_gas)

  marginal_qty = 0
  if bids['to_be_taken'][row_b] == False and asks['to_be_taken'][row_a] == False:
    marginal_qty = min(bids['qty_b'][row_b], asks['qty_a'][row_a])
  elif bids['to_be_taken'][row_b] == False:
    marginal_qty = min(sum(bids['qty_b'][:row_b+1]),sum(asks['qty_a'][:row_a+1])) - total_qty
  else:
    marginal_qty = min(sum(asks['qty_a'][:row_a+1]),sum(bids['qty_b'][:row_b+1])) - total_qty
  print('marginal_qty', marginal_qty)

  marginal_profit = marginal_qty * price_diff - marginal_gas
  print('marginal_profit', marginal_profit)
  if marginal_profit <= 0:
    break
  
  print('This order should be taken')
  total_qty += marginal_qty
  total_profit += marginal_profit
  bids.loc[row_b,'to_be_taken'] = True
  asks.loc[row_a,'to_be_taken'] = True
  
  route=(row_b,row_a)
  if math.isclose(sum(bids['qty_b'][:row_b+1]), total_qty): 
    row_b +=1
  if math.isclose(sum(asks['qty_a'][:row_a+1]), total_qty):
    row_a += 1
    

(0, 0)
price_diff 2.51
marginal_gas 2.763715906
marginal_qty 183.0
marginal_profit 456.56628409399997
nice
(0, 1)
price_diff 2.4299999999999997
marginal_gas 2.763715906
marginal_qty 17.0
marginal_profit 38.546284093999994
nice
(1, 1)


In [70]:
total_qty

200.0

In [80]:
remaining_qty = total_qty
total_collateral = 0
for i in range(max_row_b):
  if math.isclose(remaining_qty,0.0):
    break
  
  if remaining_qty < bids['qty_b'][i]:
    total_collateral += remaining_qty * bids['price_b'][i]
    bids.loc[i, 'take_percent'] = remaining_qty / bids['qty_b'][i]
    break

  total_collateral += bids['order_amt_b'][i]
  remaining_qty -= bids['qty_b'][i]
  bids.loc[i, 'take_percent'] = 1.0

In [ ]:
remaining_qty = total_qty
for i in range(max_row_a):
  if math.isclose(remaining_qty, 0.0):
    break

  if remaining_qty < asks['qty_a'][i]:
    asks.loc[i, 'take_percent'] = remaining_qty / asks['qty_a'][i]
    break

  remaining_qty -= asks['qty_a'][i]
  asks.loc[i, 'take_percent'] = 1.0

In [77]:
bids.head()

,exch_b,spread_b,raw_price_b,collateral_quoted_in_b,order_amt_b,qty_b,price_b,to_be_taken,take_percent
0,bybit,-0.02,7.72,USDT,2000.00,200.0,10.00,1.0,1.0
1,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,-900.0,-900.0
2,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,False,NaN
3,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,False,NaN
4,bybit,-0.02,7.71,USDT,15.42,2.0,7.56,False,NaN


In [62]:
asks.head()

,price_a,qty_a,order_amt_a,collateral_quoted_in_a,raw_price_a,spread_a,exch_a,to_be_taken
0,7.49,183.00,1370.15,USDT,7.30,0.03,whales,True
1,7.57,110.00,832.39,ETH,7.38,0.03,whales,True
2,7.69,34.65,266.54,USDC,7.50,0.03,whales,False
3,7.69,1350.00,10153.85,USDT,7.50,0.03,whales,False
4,7.75,1115.00,8640.11,ETH,7.56,0.03,whales,False


In [ ]:
from tg import TGSuperGroupBot
# tgs = TGSuperGroupBot(token = '7356888448:AAEyfKbHRAcWduLf_-9Cag_63J162FxI8_A', chat_id ='-1002231032838', message_thread_id=2 )
tgs.send_message(f"""
                 token: {token}
                 
                 
                 
                 """)